# Summary

Make decorator to make function handle keyboard interrupt more easily. Hoping to use this on Speaker.speak() in GUI.

UPDATE: realized I already wrote a serviceable version of this. Made a few htools tweaks, no need to use the rest of this notebook.

In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from functools import wraps
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pathlib import Path
import time

from jabberwocky.config import C
from jabberwocky.openai_utils import load_prompt, load_openai_api_key
from htools import *

In [3]:
cd_root()

Current directory: /Users/hmamin/jabberwocky


In [17]:
def interruptable(func):
    func.status_code = 0
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            res = func(*args, **kwargs)
            wrapper.status_code = 0
        except KeyboardInterrupt:
            res = RuntimeWarning('Exited early due to KeyboardInterrupt.')
            wrapper.status_code = 1
        return res
    return wrapper

In [10]:
@interruptable
def foo(n):
    res = n
    for i in range(n):
        print(i)
        time.sleep(1)
        res *= (i + 1)
    return res

In [11]:
res = foo(3)
print('res:', res)
print('status_code', foo.status_code)

0
1
2
res: 18
status_code 0


In [12]:
res = foo(3)
print('res:', res)
print('status_code', foo.status_code)

0
1
2
res: Exited early due to KeyboardInterrupt.
status_code 1


In [13]:
res

RuntimeWarning('Exited early due to KeyboardInterrupt.')

In [14]:
isinstance(res, Exception)

True

## Callbacks

Toyed with idea of integrating callbacks, but using @callback internally isn't ideal since on_end is effectively treated as part of the wrapped function's execution and therefore won't run if the function is interrupted. We could stick these inside else/finally after the try/except but I'm starting to question how useful any of this is considering the overhead of writing callbacks (an issue I should probably address).

In [44]:
class ConditionalCallback(Callback):
    
    def setup(self, func):
        pass
    
    def on_begin(self, func, inputs, output=None):
        pass
    
    def on_end(self, func, inputs, output=None):
        if func.status_code == 0:
            print('in on_end (success)')
            return output / 10
        else:
            print('in on_end (failure)')
            return -1

In [45]:
class AlwaysCallback(Callback):
    
    def setup(self, func):
        pass
    
    def on_begin(self, func, inputs, output=None):
        pass
    
    def on_end(self, func, inputs, output=None):
        print('always callback:', func.__name__)

In [52]:
def interruptable(func=None, *, cbs=()):
    if func is None: 
        return partial(interruptable, cbs=cbs)
    func.status_code = 0
    if cbs:
        func = callbacks(tolist(cbs))(func)
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            res = func(*args, **kwargs)
            wrapper.status_code = 0
        except KeyboardInterrupt:
            res = RuntimeWarning('Exited early due to KeyboardInterrupt.')
            wrapper.status_code = 1
        return res
    return wrapper

In [53]:
@interruptable(cbs=[ConditionalCallback(), AlwaysCallback()])
def foo(n):
    res = n
    for i in range(n):
        print(i)
        time.sleep(1)
        res *= (i + 1)
    print('foo res:', res)
    return res

In [48]:
res = foo(2)
print('res:', res)
print('status:', foo.status_code)

0
1
foo res: 4
in on_end (success)
always callback: foo
res: 4
status: 0


In [49]:
res = foo(2)
print('res:', res)
print('status:', foo.status_code)

0
res: Exited early due to KeyboardInterrupt.
status: 1


In [52]:
def interruptable(func=None, *, cbs=()):
    if func is None: 
        return partial(interruptable, cbs=cbs)
    func.status_code = 0
    if cbs:
        cbs = tolist(cbs)
        func = callbacks(cbs)(func)
    @wraps(func)
    def wrapper(*args, **kwargs):
        try:
            res = func(*args, **kwargs)
            wrapper.status_code = 0
        except KeyboardInterrupt:
            res = RuntimeWarning('Exited early due to KeyboardInterrupt.')
            wrapper.status_code = 1
            for cb in cbs:
                cb.on_end(func, _, res)
        return res
    return wrapper

In [53]:
@interruptable(cbs=[ConditionalCallback(), AlwaysCallback()])
def foo(n):
    res = n
    for i in range(n):
        print(i)
        time.sleep(1)
        res *= (i + 1)
    print('foo res:', res)
    return res

In [59]:
inspect.getsourcelines(foo)

(['@interruptable(cbs=[ConditionalCallback(), AlwaysCallback()])\n',
  'def foo(n):\n',
  '    res = n\n',
  '    for i in range(n):\n',
  '        print(i)\n',
  '        time.sleep(1)\n',
  '        res *= (i + 1)\n',
  "    print('foo res:', res)\n",
  '    return res\n'],
 1)